In [1]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
import pandas as pd
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup

/Users/glen/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#the file shows the fields separated by space and quoted by ", 
#and that where someone has included a quoted string in their review 
#there is a backslash before the quotes to escape them
df = pd.read_csv('data.tsv', sep=' ', quotechar='"', escapechar='\\')

In [3]:
# Split the data into training and testing set based on the splits,csv file
# Each column consist of 25,000 test samples ID, so we can separate the data into three different train/test samples
split = pd.read_csv('splits.csv', sep='\t', quotechar='"', escapechar='\\')
split.head(3)

,split_1,split_2,split_3
0,2,9245,8403
1,3,35118,40376
2,4,28666,19247


In [4]:
# Select s to determine which train and test split (S = 1 or 2 or 3)
s = 1

train_list = []
test_list  = []
# Use left join to seperate our data 
test1 = df.merge(split[['split_1']], left_on='new_id', right_on='split_1', how='inner')
test1 = test1.drop('split_1',axis = 1)
test1 = test1.reset_index(drop=True)

#Select the rows not in test1 id
train1 = df.loc[~df['new_id'].isin(test1['new_id'])]
train1 = train1.reset_index(drop=True)

# Use left join to seperate our data 
test2 = df.merge(split[['split_2']], left_on='new_id', right_on='split_2', how='inner')
test2 = test2.drop('split_2',axis = 1)
test2 = test2.reset_index(drop=True)

#Select the rows not in test1 id
train2 = df.loc[~df['new_id'].isin(test2['new_id'])]
train2 = train2.reset_index(drop=True)

# Use left join to seperate our data 
test3 = df.merge(split[['split_3']], left_on='new_id', right_on='split_3', how='inner')
test3 = test3.drop('split_3',axis = 1)
test3 = test3.reset_index(drop=True)

#Select the rows not in test1 id
train3 = df.loc[~df['new_id'].isin(test3['new_id'])]
train3 = train3.reset_index(drop=True)

train_list.append(train1)
train_list.append(train2)
train_list.append(train3)

test_list.append(test1)
test_list.append(test2)
test_list.append(test3)

train = train_list[s-1]
test = test_list[s-1]

In [5]:
from nltk.corpus import stopwords # Import the stop word list
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [6]:
# clean_review = review_to_words( train["review"])
# print(clean_review)

In [7]:
# Get the number of reviews based on the dataframe column size
num_reviews = train1["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(num_reviews):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

clean_test_reviews = [] 

for i in range(num_reviews):
#     if( (i+1) % 1000 == 0 ):
#         print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )


In [8]:
clean_train_reviews[0]

'quite enjoyed movie two reasons first gives insight world loyalism northern ireland rarely treated movies tell us republican struggle second reason performances actors thought gave honest convincing portrayals seedy underworld many people hear outside native shores entertaining ganster movie stellar performances northern irish actors cast wont move earth although may slightly open peoples eyes murky world loyalist paramilitaries'

In [9]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression

# tfv = TfidfVectorizer(min_df=3,  max_features=None, 
#         strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
#         ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
#         stop_words = 'english')

# train_data_features = tfv.fit_transform(clean_train_reviews).toarray()

# test_data_features = tfv.fit_transform(clean_test_reviews).toarray()




In [10]:
# test_data_features.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

# Numpy arrays are easy to work with, so convert the result to an 
# array
all = clean_train_reviews + clean_test_reviews
all_feature = tfv.fit_transform(all).toarray()


In [ ]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])

model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, 
                         C=1, fit_intercept=True, intercept_scaling=1.0, 
                         class_weight=None, random_state=None)

model.fit(all_feature[:num_reviews], train["sentiment"])
result = model.predict(all_feature[num_reviews:])

In [ ]:
### Save the model I train in pickle 
# Save the final model in trained_model.pkl
import pickle

# data2 = grid_sgdlogreg2
output = open('trained_model.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(model, output)

output.close()

In [ ]:
from sklearn.metrics import roc_auc_score
y_true = np.array(test1['sentiment'])
y_scores = result
roc_auc_score(y_true, y_scores)